In [ ]:
from convokit import Corpus, download

corpus = Corpus(filename=download("supreme-corpus"))

In [ ]:
corpus.print_summary_stats()

In [ ]:
corpus.get

In [14]:
df = corpus.get_utterances_dataframe()

In [ ]:
print(df.iloc[243])

In [ ]:
print(df.loc[df["meta.case_id"] == "2004_03-750"])

In [1]:
    from datasets import load_dataset

In [2]:
ds = load_dataset("./source/load_SCOTUS.py",trust_remote_code=True)
dstrain = ds["train"]
dsval = ds["validation"]


In [ ]:
print(len(dstrain)+ len(dsval))

In [ ]:
link = ds[0]["justia_link"]
print([ ex["justia_link"] for ex in ds])
print(link)

In [3]:
url = "https://supreme.justia.com/cases/federal/us/539/444/"
ds = ds["validation"]

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import requests
from selenium.webdriver.chrome.service import Service
from source.utils import get_file
from tqdm.notebook import tqdm  
import json
os.makedirs('/home/labicquette/M2/projet/data/audios', exist_ok=True)

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("prefs", {
    'download.default_directory': '/home/labicquette/M2/projet/data/audios',
    'download.prompt_for_download': False,
    'download.directory_upgrade': True,
    'safebrowsing.enabled': True,
    "download.default_directory": '/home/labicquette/M2/projet/data/audios',
    "savefile.default_directory": '/home/labicquette/M2/projet/data/audios',

})
#chrome_options.add_argument("--headless=new")

driver = webdriver.Chrome(options=chrome_options)
#print(driver.capabilities)

articles = []
for opi in tqdm(range(138,len(ds))):
    opi = ds[opi]
    url = opi["justia_link"]
    
    print(url)
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)


    try : 
        #driver.find_element(By.PARTIAL_LINK_TEXT, 'www.oyez.org/')
        #driver.find_element_by_xpath("//a[contains(@href, '/mathscinet/search/mscdoc.html')]").text
        #element = driver.find_element(by=By.XPATH, value="//a[contains(@href, 'www.oyez.org/')]")
        #element = WebDriverWait(driver, 3).until(
        #    EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'www.oyez.org/')]")))
        #print(element)
        element = WebDriverWait(driver, 10).until(
        EC.visibility_of_all_elements_located((By.XPATH, "//a[contains(@href, 'www.oyez.org/')]")))
    #)  
        if type(element) == type([]):
            element = element[0]

        href = element.get_attribute("href")
        print(href)
        driver.get(href)
    except: 
        driver.quit()
        continue
    """
        try :
            element = WebDriverWait(driver, 3).until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, 'www.oyez.org/')]"))
        )
            print(element)
        except :
            #print(element)
            driver.quit()
            continue

    driver.implicitly_wait(2)
    element.click()
    driver.implicitly_wait(2)

    """


    link = WebDriverWait(driver, 2).until(
        EC.element_to_be_clickable((By.XPATH, "//a[contains(@iframe-url, 'oral_argument_audio')]"))
    )
    #link = driver.find_element(By.XPATH, "//a[contains(@iframe-url, 'oral_argument_audio')]")
    link.click()

    outHTML = link.get_attribute("outerHTML")

    #id_audio = outHTML.split("oral_argument_audio - ")[1].split('"')[0]

    #response = requests.get("https://api.oyez.org/case_media/oral_argument_audio/"+id_audio+"/download")
    
    #file_Path = "./data/audios/train/"+get_file(opi, "SCOTUS")+".mp3"

    #if response.status_code == 200:
        #print(response)
        #with open(file_Path, 'wb') as file:
            #file.write(response.content)
        #print('File downloaded successfully')
    #else:
        #print('Failed to download file')


    driver.implicitly_wait(5)
    #TODO PROBLEM OF TAG MISMATCH
    #contents =driver.find_element(By.CLASS_NAME, "transcript")
    
    #elem = driver.find_element(By.CLASS_NAME, "transcript")
    iframe = driver.find_element(By.TAG_NAME, "iframe")
    driver.switch_to.frame(iframe)

    all_elements = driver.find_elements(By.XPATH, "//body//*")
    opi_dict = []
    # Print tag names and any text (if available) for each element
    for index, element in enumerate(all_elements):
        try:
            if element.get_attribute("class") == "transcript-turn ng-scope" :
                row = {"start-times":[],
                       "stop-times":[],
                       "text": [],
                       "person":""}
                elems = element.find_elements(By.XPATH, ".//*")
                for e in elems:
                    if e.tag_name == "h4" : 
                        row["person"] = e.get_attribute("innerHTML")
                    if e.tag_name == "p":
                        row["text"] += [e.get_attribute("innerHTML")]
                        row["start-times"] += [e.get_attribute("start-time")] 
                        row["stop-times"] += [e.get_attribute("stop-time")] 
                #print(row)
                opi_dict += [row]
        except Exception as e:
            print(f" - Could not retrieve information for element {index + 1}: {e}")



    #elem = driver.find_elements(By.CLASS_NAME, "transcript-section ng-scope")
    #print(elem)
    #print(type(elem.get_attribute("innerHTML")))

    #print(len(article))
    #print(article[0].get_attribute("transcript"))
    #transcript = article[0].get_attribute("outerHTML")
#/html/body/div[2]/div/div/article
    #file_Path = "./data/transtest/test.json"
    file_Path = "./data/transcripts/train/"+get_file(opi, "SCOTUS")+".json"
    with open(file_Path, "w") as outfile: 
        json.dump({"conversations":opi_dict}, outfile)
    

    #driver.get("https://api.oyez.org/case_media/oral_argument_audio/"+id_audio+"/download")

    driver.quit()


In [ ]:
print(outHTML.split("oral_argument_audio - ")[1].split('"')[0])

In [ ]:
print(article[0].get_attribute("innerHTML"))

In [ ]:
print(article[0].get_attribute("outerHTML"))

In [ ]:
#for a in articles:
print(articles[0][0].get_attribute("outerHTML"))

In [ ]:
"https://api.oyez.org/case_media/oral_argument_audio/23600/download"